In [1]:
import  pickle
import numpy as np
circuit_paths = [ 'random_transpiled_circuit_3400.pkl', 'circuit20_20.pkl', 'alg_transpiled_circuit_13.pkl',]

In [2]:

circuits = []
for circuit_path in circuit_paths:
    with open(circuit_path,'rb') as f:
        circuits += pickle.load(f)
circuits = circuits[1:]

In [3]:
with open('counts.pkl','rb') as f:
    counts = pickle.load(f)

In [5]:
from circuit import qiskit_to_layered_circuits
dataset = []
reals = []
for cir, count in zip(circuits[:3399] ,counts[:3399]):
    if '00000' not in count:
        continue
    ground_truth_fidelity = count['00000'] / 5000
    if ground_truth_fidelity < 0.2:
        # print('bad')
        continue

    reals.append(ground_truth_fidelity)
    circuit_info = qiskit_to_layered_circuits(cir)
    circuit_info['ground_truth_fidelity'] = ground_truth_fidelity
    dataset.append(circuit_info)

In [6]:
import random
random.shuffle(dataset)

In [7]:
print(reals)

[0.7872, 0.7868, 0.8222, 0.7686, 0.7634, 0.7728, 0.778, 0.8342, 0.816, 0.829, 0.8286, 0.8506, 0.8612, 0.8286, 0.8348, 0.8274, 0.8546, 0.8308, 0.8682, 0.83, 0.7676, 0.7024, 0.6604, 0.698, 0.6848, 0.8352, 0.6722, 0.7262, 0.7506, 0.8192, 0.7736, 0.8076, 0.8084, 0.8072, 0.7572, 0.8212, 0.804, 0.7914, 0.805, 0.7192, 0.6702, 0.7046, 0.7188, 0.7384, 0.6966, 0.7334, 0.7154, 0.7096, 0.688, 0.7008, 0.5246, 0.6146, 0.4852, 0.4646, 0.5212, 0.676, 0.5698, 0.497, 0.5176, 0.8004, 0.7898, 0.6604, 0.6486, 0.6946, 0.695, 0.7678, 0.835, 0.7576, 0.7606, 0.6668, 0.7874, 0.6608, 0.785, 0.7862, 0.6986, 0.6614, 0.6906, 0.6758, 0.8, 0.7962, 0.8674, 0.8182, 0.8334, 0.8676, 0.829, 0.8086, 0.8006, 0.7988, 0.8344, 0.7566, 0.722, 0.7592, 0.775, 0.751, 0.7422, 0.7452, 0.7598, 0.8502, 0.8118, 0.9116, 0.9198, 0.925, 0.9166, 0.9254, 0.9182, 0.9226, 0.9314, 0.9258, 0.9226, 0.8338, 0.8434, 0.793, 0.842, 0.8372, 0.827, 0.8416, 0.8432, 0.8454, 0.821, 0.7616, 0.6974, 0.6952, 0.7786, 0.7432, 0.7802, 0.809, 0.7492, 0.7516, 0.

In [8]:
train_dataset, test_dataset = dataset[:-400], dataset[-400:]
len(train_dataset), len(test_dataset), len(circuits[3399:])

(2192, 400, 530)

In [9]:
single_average_error = {0: 2.266e-4, #rb
 1:  1.699e-4,
 2:  6.082e-4,
 3:  1.686e-4,
 4:  3.998e-4,
}
couple_average_error = {(0, 1):  0.00509, # rb
                        (1, 0):  0.00509,
                        (1, 2):  0.01109,
                        (2, 1):  0.01109,
                        (2, 3):  0.0066,
                        (3, 2):  0.0066,
                        (3, 4):  0.00586,
                        (4, 3):  0.00586,
}

In [10]:

t1 = [219.4202903,250.0547864,96.55085627,214.7493552,100.0536772]
t2 = [83.5742615,81.23933506,20.20507587,44.80513087,39.62335641]
prob_meas0_prep1 = [0.054,0.0224,0.0488,0.0322,0.0402]
prob_meas1_prep0 = [0.0136,0.0104,0.0242,0.0102,0.0116]
t1_t2 = {}
for i in range(5):
    qubit_i = {}
    qubit_i['T1'] = t1[i]
    qubit_i['T2'] = t2[i]
    qubit_i['prob_meas0_prep1'] = prob_meas0_prep1[i]
    qubit_i['prob_meas1_prep0'] = prob_meas1_prep0[i]
    t1_t2[i] = qubit_i
t1_t2

{0: {'T1': 219.4202903,
  'T2': 83.5742615,
  'prob_meas0_prep1': 0.054,
  'prob_meas1_prep0': 0.0136},
 1: {'T1': 250.0547864,
  'T2': 81.23933506,
  'prob_meas0_prep1': 0.0224,
  'prob_meas1_prep0': 0.0104},
 2: {'T1': 96.55085627,
  'T2': 20.20507587,
  'prob_meas0_prep1': 0.0488,
  'prob_meas1_prep0': 0.0242},
 3: {'T1': 214.7493552,
  'T2': 44.80513087,
  'prob_meas0_prep1': 0.0322,
  'prob_meas1_prep0': 0.0102},
 4: {'T1': 100.0536772,
  'T2': 39.62335641,
  'prob_meas0_prep1': 0.0402,
  'prob_meas1_prep0': 0.0116}}

In [11]:
# ['cx', 'id', 'rz', 'sx', 'x']
gate_error = {}
for k,v in single_average_error.items():
    _gate_error = {}
    _gate_error['rz'] =  v
    _gate_error['sx'] =  v
    _gate_error['x'] =  v
    _gate_error['id'] =  v
    gate_error[tuple([k])] = _gate_error
for k,v in couple_average_error.items():
    _gate_error = {}
    _gate_error['cx'] =  v
    gate_error[k] = _gate_error
gate_error

{(0,): {'rz': 0.0002266, 'sx': 0.0002266, 'x': 0.0002266, 'id': 0.0002266},
 (1,): {'rz': 0.0001699, 'sx': 0.0001699, 'x': 0.0001699, 'id': 0.0001699},
 (2,): {'rz': 0.0006082, 'sx': 0.0006082, 'x': 0.0006082, 'id': 0.0006082},
 (3,): {'rz': 0.0001686, 'sx': 0.0001686, 'x': 0.0001686, 'id': 0.0001686},
 (4,): {'rz': 0.0003998, 'sx': 0.0003998, 'x': 0.0003998, 'id': 0.0003998},
 (0, 1): {'cx': 0.00509},
 (1, 0): {'cx': 0.00509},
 (1, 2): {'cx': 0.01109},
 (2, 1): {'cx': 0.01109},
 (2, 3): {'cx': 0.0066},
 (3, 2): {'cx': 0.0066},
 (3, 4): {'cx': 0.00586},
 (4, 3): {'cx': 0.00586}}

In [12]:
second= {}
second['qubit'] = t1_t2
second['gate'] = gate_error

In [13]:
quest_train_data = []
for cir in train_dataset:
    quest_train_data.append((cir['qiskit_circuit'].qasm(), second, cir['ground_truth_fidelity']))
with open('ques5_train.data','wb')as f:
    pickle.dump(quest_train_data, f)

In [14]:
quest_test_data = []
for cir in test_dataset:
    quest_test_data.append((cir['qiskit_circuit'].qasm(), second, cir['ground_truth_fidelity']))
with open('ques5_test.data','wb')as f:
    pickle.dump(quest_test_data, f)

In [15]:
len(quest_train_data),len(quest_test_data),

(2192, 400)

In [16]:
#pre
test_predicts = [0.39651477336883545, 0.9248540997505188, 0.46513843536376953, 0.6192243695259094, 0.35390275716781616, 0.41477906703948975, 0.5940530896186829, 0.3595989942550659, 0.34374338388442993, 0.4108111262321472, 0.27137675881385803, 0.6412668228149414, 0.6184822916984558, 0.6537385582923889, 0.5532535314559937, 0.8496066927909851, 0.29016077518463135, 0.6337668299674988, 0.4188809394836426, 0.41618895530700684, 0.6924953460693359, 0.7423111796379089, 0.2508506178855896, 0.347786545753479, 0.2922958731651306, 0.3003838062286377, 0.4769315719604492, 0.25458329916000366, 0.4327792525291443, 0.4087870717048645, 0.6605777144432068, 0.7051788568496704, 0.8189125657081604, 0.5472201704978943, 0.34930336475372314, 0.5045704245567322, 0.3814651370048523, 0.3483527898788452, 0.5384430885314941, 0.311556875705719, 0.2945302128791809, 0.4823618531227112, 0.5898663997650146, 0.33198434114456177, 0.3421841859817505, 0.739308774471283, 0.24569900333881378, 0.8379189968109131, 0.4130224585533142, 0.35164427757263184, 0.5961798429489136, 0.2633383572101593, 0.49176692962646484, 0.3725821375846863, 0.5850701928138733, 0.4027741551399231, 0.717250406742096, 0.31873035430908203, 0.6639887690544128, 0.4925735592842102, 0.38897955417633057, 0.734779417514801, 0.3231349587440491, 0.6622113585472107, 0.41227996349334717, 0.3237757086753845, 0.2589050531387329, 0.6313464045524597, 0.35282468795776367, 0.4885641932487488, 0.6279866099357605, 0.23336100578308105, 0.7461274862289429, 0.3258962631225586, 0.707981526851654, 0.6188673973083496, 0.5058308839797974, 0.4949057698249817, 0.6657010316848755, 0.6094342470169067, 0.4887436628341675, 0.2941235303878784, 0.6191422343254089, 0.28057926893234253, 0.24853724241256714, 0.7414664030075073, 0.49547773599624634, 0.5156439542770386, 0.42106908559799194, 0.7411701083183289, 0.3940569758415222, 0.7541661262512207, 0.25352150201797485, 0.866132378578186, 0.4988914728164673, 0.42050063610076904, 0.5189023017883301, 0.6774548888206482, 0.3478013873100281, 0.6442440152168274, 0.40553396940231323, 0.6907600164413452, 0.5276437997817993, 0.326360285282135, 0.6430801153182983, 0.43390101194381714, 0.3550068736076355, 0.8198662400245667, 0.31976521015167236, 0.2781848907470703, 0.41756266355514526, 0.37785667181015015, 0.2667580246925354, 0.3689848780632019, 0.23943106830120087, 0.2434205263853073, 0.7596017122268677, 0.32131141424179077, 0.35266172885894775, 0.7370074987411499, 0.6365482807159424, 0.7097122073173523, 0.8126627206802368, 0.9256664514541626, 0.23947979509830475, 0.5164889693260193, 0.5423839092254639, 0.6908291578292847, 0.37546682357788086, 0.5936647653579712, 0.7434985041618347, 0.40169888734817505, 0.749050498008728, 0.6876655220985413, 0.35559189319610596, 0.40521472692489624, 0.3263744115829468, 0.4356989860534668, 0.680202841758728, 0.33738023042678833, 0.8229120373725891, 0.21514546871185303, 0.33572304248809814, 0.3370369076728821, 0.4188516139984131, 0.3791232109069824, 0.4442984461784363, 0.2980642318725586, 0.319156289100647, 0.6100608110427856, 0.5762245655059814, 0.6184847950935364, 0.31182897090911865, 0.3789518475532532, 0.869499921798706, 0.39174771308898926, 0.3753202557563782, 0.28635966777801514, 0.3310345411300659, 0.7082961201667786, 0.8186984062194824, 0.23639439046382904, 0.27218055725097656, 0.4878283143043518, 0.696622371673584, 0.3449895977973938, 0.3449661135673523, 0.2538442611694336, 0.3997032642364502, 0.3732522130012512, 0.3397411108016968, 0.23907876014709473, 0.4018225073814392, 0.370716392993927, 0.4081984758377075, 0.6484053730964661, 0.3502519130706787, 0.6482807397842407, 0.23274369537830353, 0.32158011198043823, 0.48779451847076416, 0.5205298662185669, 0.3041575253009796, 0.4369816184043884, 0.404763400554657, 0.32155275344848633, 0.3701586127281189, 0.6158561110496521, 0.42596882581710815, 0.33127492666244507, 0.2442701905965805, 0.4098110795021057, 0.4360250234603882, 0.29495033621788025, 0.3434780240058899, 0.2385629117488861, 0.7874714732170105, 0.6109293103218079, 0.5810502171516418, 0.5464769601821899, 0.8228961229324341, 0.35552269220352173, 0.4674854874610901, 0.3393515348434448, 0.6511634588241577, 0.39741575717926025, 0.5107526779174805, 0.38051193952560425, 0.5020115971565247, 0.5133705139160156, 0.29337674379348755, 0.4708343744277954, 0.35273969173431396, 0.32928168773651123, 0.8612374067306519, 0.5824062824249268, 0.33762824535369873, 0.34455353021621704, 0.4275274872779846, 0.6742554306983948, 0.5711292624473572, 0.6133653521537781, 0.6177217960357666, 0.4799269437789917, 0.4831499457359314, 0.41604745388031006, 0.3473069667816162, 0.6938114166259766, 0.31802505254745483, 0.40397971868515015, 0.41474950313568115, 0.7126632928848267, 0.8250532746315002, 0.4077848196029663, 0.40021687746047974, 0.5423428416252136, 0.48572224378585815, 0.6715914011001587, 0.2918373942375183, 0.619403064250946, 0.35089123249053955, 0.4294682741165161, 0.5861310362815857, 0.7301344275474548, 0.23994497954845428, 0.40915077924728394, 0.4238824248313904, 0.2911473214626312, 0.32379359006881714, 0.2659284472465515, 0.34986215829849243, 0.2957131564617157, 0.7683076858520508, 0.35734641551971436, 0.8011507987976074, 0.2987906336784363, 0.5001675486564636, 0.6267067790031433, 0.5916246771812439, 0.4055207371711731, 0.2897464334964752, 0.3900691866874695, 0.6740599870681763, 0.34535735845565796, 0.854913055896759, 0.4246055483818054, 0.33210647106170654, 0.3342408537864685, 0.6517019271850586, 0.29806387424468994, 0.3476831912994385, 0.917898952960968, 0.6093167066574097, 0.3345228433609009, 0.3420376181602478, 0.3772276043891907, 0.617097795009613, 0.7986396551132202, 0.3374931812286377, 0.756358802318573, 0.3839120864868164, 0.7553223371505737, 0.6180245280265808, 0.36506229639053345, 0.37932199239730835, 0.4953327775001526, 0.38102757930755615, 0.6903066039085388, 0.37103110551834106, 0.5145384073257446, 0.514096200466156, 0.7492218613624573, 0.6029233932495117, 0.7803117036819458, 0.2862284183502197, 0.6007329225540161, 0.656756579875946, 0.36287587881088257, 0.6962001919746399, 0.7284996509552002, 0.5608349442481995, 0.7156000733375549, 0.2996966540813446, 0.6145449876785278, 0.4266062378883362, 0.5122870206832886, 0.43518155813217163, 0.3408832550048828, 0.30314958095550537, 0.7916290163993835, 0.3668822646141052, 0.486799418926239, 0.2438625693321228, 0.3331427574157715, 0.6336476802825928, 0.6626380681991577, 0.5810768604278564, 0.6371241211891174, 0.62077796459198, 0.408414363861084, 0.3342822790145874, 0.866029679775238, 0.4199790358543396, 0.41206079721450806, 0.4075967073440552, 0.6597370505332947, 0.35255759954452515, 0.26096516847610474, 0.6814336180686951, 0.3710223436355591, 0.43298304080963135, 0.2269941121339798, 0.8245041370391846, 0.3904266357421875, 0.48763537406921387, 0.39992159605026245, 0.3390192985534668, 0.2778884172439575, 0.41573846340179443, 0.7529038190841675, 0.4669625759124756, 0.6259340047836304, 0.4127100706100464, 0.4221508502960205, 0.39364802837371826, 0.6401978731155396, 0.5132663249969482, 0.6319052577018738, 0.4241871237754822, 0.34469348192214966, 0.4198121428489685, 0.38567858934402466, 0.5918962955474854, 0.40198779106140137, 0.6909785270690918, 0.48694372177124023, 0.7165949940681458, 0.30819088220596313, 0.6860042214393616, 0.6245613694190979, 0.39990556240081787, 0.5115163922309875, 0.6267067790031433, 0.39868229627609253, 0.3480297327041626, 0.6925875544548035, 0.3531057834625244, 0.6524475812911987, 0.7525884509086609, 0.4157385230064392, 0.6149770617485046, 0.27784764766693115, 0.3970809578895569, 0.33518821001052856, 0.27951574325561523, 0.5649215579032898, 0.6250579357147217, 0.741008460521698, 0.7398397326469421, 0.8507010340690613, 0.35648441314697266, 0.34196019172668457, 0.7006266117095947, 0.41553282737731934, 0.7212579846382141, 0.8199230432510376, 0.42255866527557373, 0.28629380464553833, 0.4843828082084656, 0.32637882232666016, 0.6622262597084045, 0.4238618016242981, 0.6139818429946899, 0.5597291588783264, 0.7516486644744873, 0.2523651123046875, 0.5103092789649963, 0.5459415316581726, 0.4878273606300354, 0.5100565552711487]

In [17]:

# bias
# predict = [0.37084832191467276, 0.8991876482963561, 0.43947198390960684, 0.5935579180717467, 0.3282363057136535, 0.38911261558532706, 0.5683866381645202, 0.33393254280090323, 0.31807693243026725, 0.38514467477798453, 0.24571030735969535, 0.6156003713607787, 0.5928158402442931, 0.6280721068382262, 0.527587080001831, 0.8239402413368224, 0.26449432373046866, 0.6081003785133361, 0.3932144880294799, 0.39052250385284415, 0.6668288946151733, 0.7166447281837462, 0.2251841664314269, 0.3221200942993163, 0.26662942171096793, 0.274717354774475, 0.45126512050628653, 0.22891684770584098, 0.4071128010749816, 0.3831206202507018, 0.6349112629890441, 0.6795124053955077, 0.7932461142539977, 0.5215537190437316, 0.32363691329956046, 0.4789039731025695, 0.3557986855506896, 0.32268633842468253, 0.5127766370773315, 0.2858904242515563, 0.2688637614250182, 0.4566954016685485, 0.564199948310852, 0.3063178896903991, 0.3165177345275878, 0.7136423230171203, 0.2200325518846511, 0.8122525453567504, 0.3873560070991515, 0.32597782611846915, 0.5705133914947509, 0.23767190575599662, 0.46610047817230216, 0.3469156861305236, 0.5594037413597106, 0.3771077036857604, 0.6915839552879333, 0.29306390285491934, 0.6383223176002502, 0.4669071078300475, 0.3633131027221679, 0.7091129660606383, 0.2974685072898864, 0.636544907093048, 0.3866135120391845, 0.29810925722122184, 0.23323860168457022, 0.605679953098297, 0.327158236503601, 0.4628977417945861, 0.6023201584815978, 0.20769455432891837, 0.7204610347747802, 0.3002298116683959, 0.6823150753974914, 0.5932009458541869, 0.4801644325256347, 0.469239318370819, 0.6400345802307128, 0.583767795562744, 0.4630772113800048, 0.26845707893371573, 0.5934757828712462, 0.25491281747817984, 0.22287079095840445, 0.7157999515533446, 0.46981128454208365, 0.4899775028228759, 0.39540263414382926, 0.7155036568641662, 0.36839052438735953, 0.728499674797058, 0.22785505056381217, 0.8404659271240233, 0.4732250213623046, 0.39483418464660636, 0.4932358503341674, 0.6517884373664855, 0.3221349358558654, 0.6185775637626647, 0.37986751794815055, 0.6650935649871825, 0.5019773483276366, 0.3006938338279723, 0.6174136638641357, 0.40823456048965445, 0.3293404221534728, 0.794199788570404, 0.2940987586975097, 0.2525184392929076, 0.3918962121009826, 0.35219022035598746, 0.24109157323837271, 0.3433184266090392, 0.21376461684703818, 0.21775407493114463, 0.733935260772705, 0.2956449627876281, 0.32699527740478507, 0.7113410472869872, 0.6108818292617797, 0.6840457558631896, 0.7869962692260741, 0.8999999999999999, 0.21381334364414206, 0.4908225178718566, 0.5167174577713012, 0.665162706375122, 0.3498003721237182, 0.5679983139038085, 0.717832052707672, 0.37603243589401236, 0.7233840465545653, 0.6619990706443786, 0.32992544174194327, 0.37954827547073355, 0.3007079601287841, 0.4100325345993041, 0.6545363903045653, 0.31171377897262564, 0.7972455859184264, 0.18947901725769034, 0.31005659103393546, 0.3113704562187194, 0.3931851625442504, 0.35345675945281974, 0.4186319947242736, 0.2723977804183959, 0.2934898376464843, 0.584394359588623, 0.5505581140518188, 0.5928183436393737, 0.28616251945495597, 0.3532853960990905, 0.8438334703445434, 0.36608126163482657, 0.3496538043022155, 0.26069321632385245, 0.30536808967590323, 0.6826296687126159, 0.7930319547653197, 0.21072793900966635, 0.24651410579681388, 0.4621618628501891, 0.6709559202194213, 0.3193231463432311, 0.3192996621131896, 0.2281778097152709, 0.3740368127822875, 0.34758576154708853, 0.3140746593475341, 0.21341230869293204, 0.3761560559272765, 0.3450499415397643, 0.38253202438354483, 0.6227389216423034, 0.324585461616516, 0.622614288330078, 0.20707724392414084, 0.29591366052627555, 0.4621280670166015, 0.4948634147644042, 0.27849107384681693, 0.41131516695022574, 0.3790969491004943, 0.29588630199432364, 0.3444921612739562, 0.5901896595954894, 0.40030237436294547, 0.3056084752082824, 0.21860373914241782, 0.384144628047943, 0.4103585720062255, 0.26928388476371756, 0.3178115725517272, 0.21289646029472342, 0.7618050217628478, 0.5852628588676452, 0.5553837656974792, 0.5208105087280273, 0.7972296714782714, 0.32985624074935904, 0.4418190360069274, 0.31368508338928214, 0.625497007369995, 0.37174930572509757, 0.4850862264633178, 0.35484548807144156, 0.47634514570236197, 0.48770406246185294, 0.26771029233932486, 0.4451679229736327, 0.3270732402801513, 0.30361523628234854, 0.8355709552764892, 0.5567398309707641, 0.31196179389953604, 0.31888707876205435, 0.40186103582382193, 0.6485889792442321, 0.5454628109931945, 0.5876989006996154, 0.5920553445816039, 0.454260492324829, 0.4574834942817687, 0.39038100242614737, 0.3216405153274535, 0.6681449651718139, 0.29235860109329215, 0.37831326723098746, 0.38908305168151847, 0.686996841430664, 0.7993868231773376, 0.3821183681488036, 0.37455042600631705, 0.5166763901710509, 0.46005579233169547, 0.645924949645996, 0.2661709427833556, 0.5937366127967834, 0.32522478103637686, 0.4038018226623534, 0.560464584827423, 0.7044679760932921, 0.2142785280942916, 0.38348432779312125, 0.3982159733772277, 0.26548087000846854, 0.29812713861465445, 0.24026199579238883, 0.32419570684432975, 0.270046705007553, 0.7426412343978881, 0.33167996406555167, 0.7754843473434447, 0.2731241822242736, 0.47450109720230094, 0.6010403275489806, 0.5659582257270812, 0.3798542857170104, 0.26407998204231253, 0.3644027352333068, 0.6483935356140136, 0.3196909070014953, 0.8292466044425963, 0.39893909692764273, 0.30644001960754386, 0.3085744023323058, 0.6260354757308959, 0.27239742279052725, 0.3220167398452758, 0.8922325015068053, 0.583650255203247, 0.3088563919067382, 0.3163711667060851, 0.351561152935028, 0.5914313435554504, 0.7729732036590575, 0.311826729774475, 0.7306923508644103, 0.3582456350326537, 0.729655885696411, 0.5923580765724181, 0.33939584493637076, 0.35365554094314566, 0.4696663260459899, 0.35536112785339347, 0.6646401524543761, 0.3453646540641784, 0.48887195587158194, 0.4884297490119933, 0.7235554099082946, 0.577256941795349, 0.7546452522277831, 0.26056196689605704, 0.5750664710998534, 0.6310901284217834, 0.3372094273567199, 0.6705337405204772, 0.7028331995010375, 0.5351684927940368, 0.6899336218833922, 0.2740302026271819, 0.5888785362243651, 0.4009397864341735, 0.4866205692291259, 0.40951510667800894, 0.3152168035507201, 0.2774831295013427, 0.7659625649452209, 0.34121581315994254, 0.4611329674720763, 0.21819611787796012, 0.3074763059616088, 0.6079812288284301, 0.636971616744995, 0.5554104089736938, 0.6114576697349547, 0.5951115131378173, 0.3827479124069213, 0.3086158275604247, 0.8403632283210754, 0.3943125844001769, 0.38639434576034537, 0.3819302558898925, 0.634070599079132, 0.32689114809036246, 0.23529871702194205, 0.6557671666145324, 0.3453558921813964, 0.40731658935546866, 0.2013276606798171, 0.7988376855850219, 0.3647601842880248, 0.4619689226150512, 0.37425514459609976, 0.3133528470993041, 0.25222196578979483, 0.39007201194763175, 0.7272373676300048, 0.4412961244583129, 0.6002675533294677, 0.3870436191558837, 0.3964843988418578, 0.3679815769195556, 0.6145314216613769, 0.48759987354278556, 0.6062388062477111, 0.3985206723213195, 0.31902703046798697, 0.3941456913948058, 0.36001213788986197, 0.5662298440933227, 0.3763213396072387, 0.6653120756149291, 0.46127727031707755, 0.6909285426139831, 0.28252443075180045, 0.6603377699851989, 0.5988949179649352, 0.3742391109466552, 0.48584994077682486, 0.6010403275489806, 0.37301584482192984, 0.3223632812499999, 0.6669211030006408, 0.3274393320083617, 0.626781129837036, 0.7269219994544982, 0.3900720715522765, 0.589310610294342, 0.25218119621276847, 0.3714145064353942, 0.3095217585563659, 0.25384929180145255, 0.5392551064491271, 0.599391484260559, 0.7153420090675353, 0.7141732811927795, 0.8250345826148986, 0.33081796169280997, 0.3162937402725219, 0.674960160255432, 0.38986637592315665, 0.6955915331840514, 0.7942565917968749, 0.39689221382141104, 0.26062735319137564, 0.4587163567543029, 0.30071237087249747, 0.6365598082542419, 0.3981953501701354, 0.5883153915405273, 0.5340627074241637, 0.7259822130203246, 0.2266986608505248, 0.48464282751083365, 0.5202750802040099, 0.4621609091758727, 0.484390103816986]

In [18]:
# [0.7872, 0.7868, 0.8222, 0.7686, 0.7634, 0.7728, 0.778, 0.8342, 0.816, 0.829, 0.8286, 0.8506, 0.8612, 0.8286, 0.8348, 0.8274, 0.8546, 0.8308, 0.8682, 0.83, 0.7676, 0.7024, 0.6604, 0.698, 0.6848, 0.8352, 0.6722, 0.7262, 0.7506, 0.8192, 0.7736, 0.8076, 0.8084, 0.8072, 0.7572, 0.8212, 0.804, 0.7914, 0.805, 0.7192, 0.6702, 0.7046, 0.7188, 0.7384, 0.6966, 0.7334, 0.7154, 0.7096, 0.688, 0.7008, 0.5246, 0.6146, 0.4852, 0.4646, 0.5212, 0.676, 0.5698, 0.497, 0.5176, 0.8004, 0.7898, 0.6604, 0.6486, 0.6946, 0.695, 0.7678, 0.835, 0.7576, 0.7606, 0.6668, 0.7874, 0.6608, 0.785, 0.7862, 0.6986, 0.6614, 0.6906, 0.6758, 0.8, 0.7962, 0.8674, 0.8182, 0.8334, 0.8676, 0.829, 0.8086, 0.8006, 0.7988, 0.8344, 0.7566, 0.722, 0.7592, 0.775, 0.751, 0.7422, 0.7452, 0.7598, 0.8502, 0.8118, 0.9116, 0.9198, 0.925, 0.9166, 0.9254, 0.9182, 0.9226, 0.9314, 0.9258, 0.9226, 0.8338, 0.8434, 0.793, 0.842, 0.8372, 0.827, 0.8416, 0.8432, 0.8454, 0.821, 0.7616, 0.6974, 0.6952, 0.7786, 0.7432, 0.7802, 0.809, 0.7492, 0.7516, 0.7772, 0.9052, 0.9112, 0.9026, 0.9116, 0.9146, 0.9028, 0.9008, 0.909, 0.9192, 0.8986, 0.73, 0.7864, 0.8104, 0.747, 0.8302, 0.8184, 0.7604, 0.8228, 0.7374, 0.749, 0.6712, 0.7024, 0.7056, 0.6872, 0.6232, 0.7868, 0.6464, 0.6774, 0.6878, 0.7494, 0.899, 0.8944, 0.8868, 0.8948, 0.893, 0.8798, 0.897, 0.8872, 0.8898, 0.8852, 0.8812, 0.8842, 0.8796, 0.8796, 0.892, 0.8698, 0.8794, 0.8704, 0.9002, 0.8886, 0.8702, 0.8694, 0.8836, 0.8682, 0.8756, 0.8618, 0.8744, 0.886, 0.8712, 0.8828, 0.8528, 0.8646, 0.877, 0.8768, 0.8724, 0.8624, 0.8734, 0.8412, 0.8538, 0.8744, 0.6114, 0.5702, 0.498, 0.6636, 0.5974, 0.5108, 0.5294, 0.7638, 0.495, 0.6532, 0.7026, 0.6694, 0.6944, 0.753, 0.7554, 0.7974, 0.7596, 0.7474, 0.7656, 0.7694, 0.7244, 0.6594, 0.7014, 0.64, 0.5772, 0.5826, 0.6844, 0.6438, 0.7286, 0.6226, 0.8768, 0.8778, 0.8716, 0.8854, 0.8782, 0.8798, 0.8662, 0.8756, 0.8748, 0.8714, 0.7856, 0.79, 0.7132, 0.7, 0.7204, 0.7672, 0.7388, 0.6728, 0.6416, 0.7382, 0.6818, 0.5864, 0.7096, 0.7482, 0.5804, 0.601, 0.6864, 0.6882, 0.798, 0.6934, 0.7028, 0.6656, 0.6612, 0.7016, 0.7002, 0.676, 0.6836, 0.8008, 0.7112, 0.6756, 0.7916, 0.7618, 0.782, 0.787, 0.7534, 0.7828, 0.7488, 0.7978, 0.7662, 0.8056, 0.755, 0.776, 0.7736, 0.7494, 0.7584, 0.7382, 0.7606, 0.7978, 0.7638, 0.7358, 0.7982, 0.7424, 0.7156, 0.7552, 0.788, 0.7678, 0.7546, 0.7996, 0.7462, 0.7328, 0.5882, 0.648, 0.3466, 0.6592, 0.383, 0.3348, 0.6456, 0.504, 0.5162, 0.617, 0.8138, 0.7988, 0.83, 0.8124, 0.7798, 0.823, 0.8182, 0.7266, 0.809, 0.8128, 0.8298, 0.799, 0.7842, 0.8256, 0.7954, 0.7898, 0.7952, 0.7884, 0.8178, 0.8012, 0.7476, 0.7546, 0.7592, 0.757, 0.7838, 0.7616, 0.7636, 0.7678, 0.729, 0.8048, 0.7086, 0.6862, 0.7154, 0.6906, 0.7026, 0.6942, 0.7904, 0.684, 0.7674, 0.6382, 0.6758, 0.6488, 0.6062, 0.6286, 0.6554, 0.635, 0.6316, 0.6126, 0.6454, 0.6156, 0.7404, 0.806, 0.7974, 0.7766, 0.782, 0.8194, 0.7848, 0.8214, 0.6632, 0.7888, 0.746, 0.8228, 0.7494, 0.7392, 0.8214, 0.757, 0.7454, 0.7278, 0.7556, 0.745, 0.5282, 0.5926, 0.6916, 0.4056, 0.3954, 0.57, 0.5958, 0.3866, 0.3184, 0.661, 0.8304, 0.8228, 0.7952, 0.825, 0.8452, 0.7828, 0.8304, 0.808, 0.8092, 0.8052, 0.8496, 0.8282, 0.839, 0.844, 0.8382, 0.8326, 0.8224, 0.8352, 0.835, 0.823, 0.686, 0.7826, 0.6736, 0.656, 0.6544, 0.6498, 0.6566, 0.6456, 0.6606, 0.7704, 0.548, 0.6608, 0.6782, 0.6678, 0.574, 0.6614, 0.5752, 0.5378, 0.5316, 0.5382, 0.7316, 0.7052, 0.7236, 0.8188, 0.7562, 0.7406, 0.7432, 0.723, 0.7984, 0.7478, 0.6892, 0.6586, 0.6292, 0.685, 0.7212, 0.6184, 0.6484, 0.71, 0.6582, 0.6712, 0.7522, 0.7802, 0.7282, 0.7594, 0.7722, 0.768, 0.7886, 0.7658, 0.7648, 0.778, 0.571, 0.511, 0.664, 0.5706, 0.5898, 0.509, 0.5708, 0.5522, 0.4988, 0.514, 0.6302, 0.841, 0.6138, 0.6016, 0.8246, 0.8194, 0.6624, 0.5896, 0.5996, 0.805, 0.7524, 0.7492, 0.7178, 0.7342, 0.7236, 0.7728, 0.7298, 0.7026, 0.7666, 0.7118, 0.6784, 0.6288, 0.638, 0.711, 0.6524, 0.7064, 0.7148, 0.655, 0.6282, 0.633, 0.6232, 0.676, 0.636, 0.5536, 0.6426, 0.5442, 0.6962, 0.56, 0.6058, 0.5262, 0.7616, 0.768, 0.5764, 0.6962, 0.6738, 0.7438, 0.689, 0.6978, 0.67, 0.653, 0.3274, 0.4002, 0.3722, 0.3958, 0.4228, 0.2822, 0.4076, 0.4214, 0.3244, 0.2798, 0.666, 0.6924, 0.7456, 0.7112, 0.6796, 0.6612, 0.6978, 0.7218, 0.6892, 0.7668, 0.6326, 0.5352, 0.5486, 0.5376, 0.6726, 0.615, 0.6414, 0.629, 0.535, 0.6248, 0.5576, 0.5054, 0.4662, 0.4476, 0.5222, 0.5176, 0.5498, 0.3966, 0.4202, 0.5286, 0.705, 0.777, 0.762, 0.7362, 0.7474, 0.7358, 0.7366, 0.7402, 0.7556, 0.747, 0.7222, 0.749, 0.7256, 0.7144, 0.7124, 0.6832, 0.7056, 0.7004, 0.7034, 0.748, 0.7402, 0.7602, 0.7024, 0.7434, 0.7404, 0.7448, 0.7418, 0.7626, 0.777, 0.7286, 0.7644, 0.6034, 0.7424, 0.6418, 0.6944, 0.7852, 0.5898, 0.6618, 0.7242, 0.562, 0.7866, 0.74, 0.7956, 0.7724, 0.754, 0.7722, 0.7834, 0.7538, 0.7552, 0.7554, 0.4884, 0.5614, 0.5466, 0.5988, 0.6388, 0.5242, 0.521, 0.5088, 0.5472, 0.6292, 0.6326, 0.5506, 0.6374, 0.5782, 0.6032, 0.5478, 0.563, 0.5848, 0.5642, 0.5864, 0.6078, 0.5212, 0.57, 0.6092, 0.5332, 0.513, 0.503, 0.6128, 0.5688, 0.4892, 0.4986, 0.6496, 0.3358, 0.5348, 0.579, 0.495, 0.5946, 0.5546, 0.4292, 0.2622, 0.4984, 0.5482, 0.5182, 0.5998, 0.4722, 0.5352, 0.5344, 0.5306, 0.5448, 0.475, 0.4772, 0.4772, 0.497, 0.3632, 0.4556, 0.4508, 0.4324, 0.546, 0.4678, 0.4992, 0.5684, 0.6196, 0.5368, 0.6532, 0.7376, 0.6298, 0.57, 0.6162, 0.537, 0.5442, 0.6334, 0.6322, 0.6122, 0.5896, 0.6784, 0.6204, 0.6732, 0.6252, 0.6072, 0.6224, 0.531, 0.4698, 0.5068, 0.5552, 0.5684, 0.5098, 0.4968, 0.5442, 0.555, 0.6396, 0.772, 0.7496, 0.761, 0.772, 0.7508, 0.7476, 0.7508, 0.7434, 0.7776, 0.7468, 0.7654, 0.7464, 0.7614, 0.7106, 0.739, 0.7002, 0.764, 0.693, 0.7484, 0.7478, 0.5348, 0.536, 0.4662, 0.5316, 0.4804, 0.4406, 0.4982, 0.4628, 0.5352, 0.4542, 0.4618, 0.5072, 0.4706, 0.6302, 0.5758, 0.5378, 0.4912, 0.423, 0.4962, 0.5246, 0.7158, 0.7858, 0.785, 0.7678, 0.7718, 0.7404, 0.7578, 0.7364, 0.7322, 0.7304, 0.605, 0.6076, 0.6476, 0.6516, 0.5842, 0.5708, 0.538, 0.66, 0.6022, 0.5678, 0.534, 0.4616, 0.465, 0.4776, 0.4618, 0.583, 0.5246, 0.4788, 0.5966, 0.6802, 0.6502, 0.6456, 0.658, 0.6566, 0.6356, 0.675, 0.6364, 0.689, 0.6352, 0.6228, 0.565, 0.5962, 0.668, 0.702, 0.3872, 0.5768, 0.6686, 0.4766, 0.5298, 0.3666, 0.541, 0.5688, 0.391, 0.6836, 0.6242, 0.705, 0.4368, 0.3994, 0.5538, 0.4298, 0.401, 0.3672, 0.4662, 0.4036, 0.3662, 0.3582, 0.6468, 0.6636, 0.548, 0.642, 0.7154, 0.6006, 0.6678, 0.6402, 0.608, 0.6744, 0.59, 0.5588, 0.6066, 0.5898, 0.2404, 0.2622, 0.2978, 0.326, 0.21, 0.3528, 0.6632, 0.694, 0.655, 0.6844, 0.7138, 0.7084, 0.6596, 0.6948, 0.724, 0.746, 0.7458, 0.7158, 0.7318, 0.7758, 0.7058, 0.689, 0.721, 0.7468, 0.7316, 0.7524, 0.5852, 0.5, 0.4436, 0.4916, 0.5206, 0.5508, 0.6206, 0.5554, 0.5204, 0.5088, 0.5702, 0.518, 0.6268, 0.586, 0.5686, 0.5482, 0.5894, 0.5666, 0.6114, 0.5848, 0.5098, 0.3968, 0.4578, 0.4764, 0.438, 0.4146, 0.444, 0.3366, 0.4184, 0.4786, 0.5318, 0.6294, 0.5242, 0.599, 0.5126, 0.5866, 0.6112, 0.496, 0.5708, 0.611, 0.4912, 0.2374, 0.2392, 0.3148, 0.311, 0.4466, 0.257, 0.5614, 0.5724, 0.3888, 0.706, 0.6668, 0.6074, 0.6812, 0.7144, 0.6614, 0.5918, 0.6226, 0.7166, 0.7424, 0.5232, 0.7268, 0.5984, 0.5802, 0.587, 0.5932, 0.6568, 0.5462, 0.5398, 0.5244, 0.3558, 0.2496, 0.2766, 0.219, 0.4396, 0.3258, 0.3288, 0.4112, 0.2896, 0.7864, 0.7658, 0.7812, 0.7438, 0.7794, 0.7788, 0.7608, 0.7604, 0.7694, 0.7858, 0.7292, 0.7346, 0.7642, 0.7426, 0.728, 0.7854, 0.7462, 0.745, 0.7614, 0.763, 0.5792, 0.3724, 0.5556, 0.3938, 0.4486, 0.4502, 0.3976, 0.489, 0.4232, 0.582, 0.5624, 0.592, 0.6424, 0.6126, 0.657, 0.6178, 0.5782, 0.6222, 0.594, 0.5762, 0.5206, 0.5124, 0.6658, 0.456, 0.5356, 0.5862, 0.5198, 0.5116, 0.542, 0.5038, 0.6636, 0.6438, 0.6222, 0.6308, 0.568, 0.5736, 0.6086, 0.5894, 0.5308, 0.6576, 0.3584, 0.3336, 0.4608, 0.4192, 0.3548, 0.359, 0.377, 0.375, 0.3678, 0.4848, 0.6912, 0.7066, 0.6044, 0.6008, 0.5508, 0.6214, 0.6366, 0.627, 0.6948, 0.6106, 0.3366, 0.3126, 0.3032, 0.3536, 0.3558, 0.2896, 0.3256, 0.397, 0.4402, 0.4278, 0.4992, 0.4976, 0.4896, 0.4676, 0.4598, 0.5452, 0.5046, 0.4958, 0.6394, 0.4804, 0.6796, 0.707, 0.652, 0.6836, 0.7068, 0.6528, 0.6702, 0.664, 0.7036, 0.6604, 0.3326, 0.2976, 0.335, 0.2718, 0.3018, 0.3578, 0.2824, 0.347, 0.2532, 0.296, 0.744, 0.717, 0.7114, 0.7284, 0.706, 0.7276, 0.7228, 0.6998, 0.6898, 0.7006, 0.505, 0.3842, 0.4318, 0.5124, 0.513, 0.576, 0.5302, 0.5656, 0.466, 0.4676, 0.4602, 0.2048, 0.2714, 0.2404, 0.5356, 0.507, 0.5708, 0.4822, 0.5248, 0.4708, 0.5144, 0.5312, 0.5136, 0.5212, 0.5988, 0.5374, 0.682, 0.6354, 0.5836, 0.5102, 0.607, 0.6534, 0.5386, 0.527, 0.2274, 0.2102, 0.259, 0.3072, 0.2334, 0.2194, 0.2474, 0.334, 0.2204, 0.4176, 0.5358, 0.5188, 0.3446, 0.4688, 0.3676, 0.5082, 0.3806, 0.4844, 0.3688, 0.4068, 0.269, 0.4906, 0.2228, 0.3238, 0.3858, 0.3326, 0.2886, 0.2004, 0.2898, 0.3196, 0.4394, 0.2894, 0.3572, 0.4604, 0.33, 0.4602, 0.4464, 0.4698, 0.419, 0.7056, 0.5406, 0.56, 0.606, 0.6544, 0.7222, 0.68, 0.5622, 0.5898, 0.5746, 0.4478, 0.5236, 0.5854, 0.4356, 0.5008, 0.5654, 0.4978, 0.4676, 0.4306, 0.4774, 0.4244, 0.4036, 0.297, 0.407, 0.3836, 0.426, 0.334, 0.3506, 0.5138, 0.4042, 0.5874, 0.4694, 0.5824, 0.6206, 0.5408, 0.5198, 0.5132, 0.5312, 0.5966, 0.6008, 0.67, 0.6206, 0.6442, 0.6396, 0.6762, 0.6282, 0.645, 0.7084, 0.6546, 0.6294, 0.5096, 0.6072, 0.5406, 0.5556, 0.5374, 0.576, 0.5088, 0.5184, 0.5644, 0.4792, 0.2496, 0.2472, 0.2312, 0.2954, 0.2584, 0.2104, 0.2854, 0.2194, 0.2914, 0.2852, 0.3846, 0.3718, 0.383, 0.4066, 0.413, 0.394, 0.3672, 0.4272, 0.3836, 0.3538, 0.2546, 0.2036, 0.2918, 0.3542, 0.3316, 0.4034, 0.369, 0.2188, 0.2086, 0.3218, 0.7154, 0.6746, 0.6802, 0.7056, 0.706, 0.6878, 0.68, 0.6884, 0.6986, 0.7132, 0.652, 0.65, 0.6456, 0.6354, 0.6724, 0.6784, 0.6342, 0.658, 0.671, 0.6586, 0.3284, 0.4974, 0.5272, 0.2992, 0.361, 0.5126, 0.4022, 0.5004, 0.3234, 0.3266, 0.3622, 0.4434, 0.2248, 0.2716, 0.251, 0.227, 0.2128, 0.4234, 0.275, 0.3764, 0.3986, 0.3706, 0.2458, 0.3248, 0.4516, 0.5642, 0.2608, 0.3696, 0.3514, 0.241, 0.2124, 0.3578, 0.312, 0.223, 0.2062, 0.203, 0.3266, 0.5246, 0.5256, 0.613, 0.6392, 0.591, 0.6652, 0.6316, 0.5534, 0.574, 0.5456, 0.4598, 0.6646, 0.4162, 0.4396, 0.4718, 0.5246, 0.4754, 0.5862, 0.4974, 0.5756, 0.3896, 0.2812, 0.265, 0.2838, 0.3004, 0.3142, 0.4402, 0.2074, 0.4706, 0.4534, 0.2876, 0.3222, 0.2284, 0.2898, 0.4606, 0.345, 0.3388, 0.394, 0.3488, 0.2762, 0.4262, 0.2836, 0.3412, 0.3886, 0.2568, 0.2936, 0.2404, 0.2266, 0.3078, 0.2304, 0.2728, 0.284, 0.2558, 0.352, 0.2506, 0.6074, 0.614, 0.6878, 0.653, 0.6862, 0.6594, 0.617, 0.7226, 0.6512, 0.7226, 0.444, 0.3996, 0.3882, 0.4262, 0.4306, 0.4714, 0.4618, 0.485, 0.4928, 0.435, 0.6194, 0.5994, 0.5268, 0.567, 0.5986, 0.4974, 0.5412, 0.586, 0.607, 0.598, 0.2654, 0.2706, 0.269, 0.2376, 0.2812, 0.2144, 0.2122, 0.216, 0.263, 0.2412, 0.3978, 0.4366, 0.3652, 0.2992, 0.3254, 0.2896, 0.2464, 0.5322, 0.2284, 0.4158, 0.4006, 0.2966, 0.2368, 0.4998, 0.388, 0.3674, 0.3394, 0.491, 0.3144, 0.259, 0.2556, 0.2732, 0.2658, 0.382, 0.2926, 0.2698, 0.2316, 0.2524, 0.24, 0.3022, 0.5096, 0.4712, 0.4378, 0.453, 0.4996, 0.4154, 0.4524, 0.513, 0.5208, 0.4932, 0.2248, 0.6632, 0.6028, 0.6224, 0.6048, 0.6474, 0.6334, 0.6618, 0.641, 0.6508, 0.6504, 0.4714, 0.4182, 0.595, 0.3594, 0.4486, 0.4578, 0.4486, 0.4608, 0.4308, 0.4642, 0.5244, 0.574, 0.5724, 0.6192, 0.628, 0.6102, 0.6188, 0.6352, 0.5186, 0.727, 0.374, 0.5312, 0.3552, 0.3538, 0.3566, 0.447, 0.3488, 0.4058, 0.4494, 0.4244, 0.4866, 0.44, 0.4908, 0.4634, 0.4782, 0.4324, 0.3304, 0.4134, 0.4194, 0.3876, 0.4576, 0.2812, 0.489, 0.314, 0.5142, 0.3302, 0.47, 0.3862, 0.3942, 0.281, 0.2474, 0.3722, 0.5036, 0.3614, 0.3398, 0.396, 0.245, 0.3856, 0.2206, 0.4102, 0.2246, 0.2454, 0.2182, 0.556, 0.4696, 0.5108, 0.5588, 0.4566, 0.4676, 0.4044, 0.546, 0.5138, 0.3568, 0.2826, 0.33, 0.3028, 0.3006, 0.3038, 0.3364, 0.265, 0.2898, 0.3082, 0.2888, 0.2768, 0.2988, 0.3576, 0.3118, 0.477, 0.251, 0.3232, 0.332, 0.3326, 0.3248, 0.299, 0.216, 0.2274, 0.2688, 0.2354, 0.34, 0.303, 0.324, 0.4028, 0.4818, 0.2506, 0.3838, 0.3862, 0.3158, 0.2864, 0.3088, 0.4664, 0.3608, 0.4086, 0.3934, 0.3774, 0.4106, 0.5918, 0.6446, 0.5502, 0.662, 0.6028, 0.6328, 0.5348, 0.6722, 0.5884, 0.62, 0.248, 0.2534, 0.2264, 0.2312, 0.2394, 0.4762, 0.4342, 0.501, 0.493, 0.4548, 0.4508, 0.4556, 0.4854, 0.4614, 0.4688, 0.215, 0.2264, 0.4076, 0.4352, 0.4662, 0.5078, 0.4368, 0.4438, 0.4378, 0.5136, 0.4634, 0.4448, 0.3326, 0.276, 0.3362, 0.3834, 0.3696, 0.3504, 0.3558, 0.3112, 0.3212, 0.3356, 0.2712, 0.3498, 0.2774, 0.2848, 0.3304, 0.3018, 0.2958, 0.3168, 0.2982, 0.2314, 0.4768, 0.5474, 0.4294, 0.4244, 0.433, 0.3918, 0.4846, 0.5414, 0.374, 0.393, 0.3402, 0.3954, 0.4712, 0.2294, 0.2658, 0.3056, 0.2618, 0.43, 0.2726, 0.3972, 0.3012, 0.2796, 0.3302, 0.318, 0.3012, 0.3346, 0.3138, 0.3472, 0.364, 0.3258, 0.4726, 0.441, 0.4468, 0.5098, 0.427, 0.4698, 0.4766, 0.4358, 0.4368, 0.4854, 0.4838, 0.3118, 0.3658, 0.4576, 0.3702, 0.4636, 0.389, 0.452, 0.3114, 0.2952, 0.4502, 0.269, 0.3312, 0.3662, 0.566, 0.2176, 0.4096, 0.3706, 0.2694, 0.4476, 0.2058, 0.394, 0.2032, 0.3932, 0.3694, 0.3716, 0.4022, 0.4296, 0.4256, 0.3668, 0.492, 0.4098, 0.2964, 0.3328, 0.4018, 0.4322, 0.4702, 0.3546, 0.3586, 0.4188, 0.3412, 0.3494, 0.348, 0.359, 0.3504, 0.2948, 0.3216, 0.2934, 0.5018, 0.3852, 0.4584, 0.352, 0.3618, 0.3644, 0.3138, 0.3124, 0.4484, 0.5008, 0.2264, 0.2272, 0.2806, 0.4164, 0.3542, 0.2302, 0.252, 0.2084, 0.3796, 0.2366, 0.2988, 0.2416, 0.2664, 0.2574, 0.2116, 0.271, 0.3056, 0.3428, 0.3288, 0.347, 0.384, 0.431, 0.3732, 0.3552, 0.3362, 0.3234, 0.3734, 0.3046, 0.271, 0.3552, 0.3092, 0.29, 0.2992, 0.2942, 0.2656, 0.275, 0.246, 0.5416, 0.3278, 0.3002, 0.2974, 0.5456, 0.2998, 0.4428, 0.3282, 0.517, 0.471, 0.5096, 0.6152, 0.5046, 0.5076, 0.4358, 0.624, 0.567, 0.5962, 0.4574, 0.5548, 0.295, 0.3122, 0.363, 0.4416, 0.3772, 0.3494, 0.3788, 0.2848, 0.3906, 0.315, 0.4238, 0.4974, 0.5106, 0.438, 0.349, 0.4976, 0.3468, 0.4648, 0.446, 0.5398, 0.5282, 0.5354, 0.4974, 0.5518, 0.523, 0.5582, 0.5204, 0.539, 0.5, 0.5612, 0.2124, 0.29, 0.2748, 0.2944, 0.2216, 0.3462, 0.2414, 0.2324, 0.375, 0.2308, 0.2882, 0.4488, 0.4866, 0.5046, 0.4416, 0.3812, 0.6254, 0.5094, 0.4912, 0.4118, 0.5142, 0.3956, 0.4522, 0.398, 0.4032, 0.431, 0.4652, 0.4548, 0.3576, 0.3888, 0.3824, 0.4224, 0.4726, 0.3902, 0.382, 0.4234, 0.4542, 0.4484, 0.3576, 0.3938, 0.5188, 0.3178, 0.4672, 0.399, 0.324, 0.4494, 0.5668, 0.3376, 0.3724, 0.2246, 0.3084, 0.3314, 0.2818, 0.3562, 0.3484, 0.2906, 0.388, 0.295, 0.358, 0.3168, 0.4492, 0.3652, 0.3454, 0.2826, 0.264, 0.3378, 0.2176, 0.6428, 0.634, 0.5216, 0.6654, 0.5238, 0.5048, 0.5286, 0.5654, 0.557, 0.5572, 0.2712, 0.4466, 0.4674, 0.4478, 0.4816, 0.4722, 0.4604, 0.4726, 0.4842, 0.4176, 0.4768, 0.3672, 0.4096, 0.3404, 0.2074, 0.2978, 0.229, 0.3118, 0.467, 0.3518, 0.481, 0.5478, 0.5326, 0.537, 0.4556, 0.5452, 0.5112, 0.4892, 0.5408, 0.4982, 0.4538, 0.3436, 0.2104, 0.2976, 0.3226, 0.3838, 0.351, 0.4018, 0.3088, 0.3914, 0.275, 0.2616, 0.2974, 0.2588, 0.3342, 0.2434, 0.2724, 0.2998, 0.4084, 0.2692, 0.3072, 0.313, 0.3552, 0.4944, 0.3954, 0.492, 0.4718, 0.4238, 0.4246, 0.5084, 0.4496, 0.4036, 0.5354, 0.2346, 0.2066, 0.2226, 0.2208, 0.2092, 0.351, 0.3746, 0.3236, 0.3708, 0.3582, 0.3786, 0.225, 0.2926, 0.2558, 0.2586, 0.3434, 0.4034, 0.4084, 0.2694, 0.3288, 0.3394, 0.3562, 0.4192, 0.383, 0.3258, 0.2366, 0.2418, 0.207, 0.222, 0.2508, 0.2456, 0.2694, 0.301, 0.3062, 0.2686, 0.2526, 0.2928, 0.2854, 0.234, 0.3518, 0.205, 0.2834, 0.3524, 0.2354, 0.2636, 0.2666, 0.2058, 0.533, 0.3818, 0.232, 0.3216, 0.2176, 0.233, 0.2874, 0.4564, 0.2636, 0.263, 0.4966, 0.4132, 0.4452, 0.2792, 0.446, 0.2354, 0.361, 0.2478, 0.2228, 0.226, 0.2212, 0.3864, 0.4518, 0.4162, 0.3976, 0.3102, 0.346, 0.3722, 0.3256, 0.3146, 0.2926, 0.2712, 0.4152, 0.4026, 0.2992, 0.3738, 0.3816, 0.3858, 0.4254, 0.5134, 0.443, 0.4184, 0.491, 0.4306, 0.3586, 0.4346, 0.4414, 0.4454, 0.4188, 0.4818, 0.4618, 0.4282, 0.2204, 0.2134, 0.2642, 0.2336, 0.2236, 0.2106, 0.3928, 0.3798, 0.4286, 0.4432, 0.4932, 0.3534, 0.3482, 0.3942, 0.395, 0.443, 0.2588, 0.3502, 0.3934, 0.2164, 0.378, 0.5322, 0.519, 0.5682, 0.5618, 0.5456, 0.4988, 0.458, 0.514, 0.504, 0.5256, 0.3318, 0.3172, 0.4148, 0.2884, 0.319, 0.3024, 0.3106, 0.3782, 0.3482, 0.2862, 0.3134, 0.293, 0.2472, 0.2156, 0.2592, 0.2236, 0.2544, 0.3074, 0.2292, 0.296, 0.3994, 0.4536, 0.4284, 0.4646, 0.424, 0.5098, 0.43, 0.4904, 0.4476, 0.4752, 0.208, 0.2374, 0.2446, 0.23, 0.2874, 0.3934, 0.3894, 0.3198, 0.3974, 0.3522, 0.4422, 0.4216, 0.3386, 0.4048, 0.3768, 0.2146, 0.3116, 0.205, 0.3384, 0.3714, 0.3138, 0.4148, 0.2812, 0.3182, 0.2914, 0.3238, 0.3494, 0.3358, 0.2104, 0.21, 0.218, 0.26, 0.265, 0.2638, 0.2154, 0.2556, 0.2566, 0.2436, 0.2502, 0.2416, 0.3286, 0.3602, 0.3258, 0.3512, 0.3304, 0.325, 0.3472, 0.3152, 0.3594, 0.361, 0.4616, 0.445, 0.3426, 0.4134, 0.4626, 0.3726, 0.2812, 0.3446, 0.3892, 0.355, 0.2556, 0.2774, 0.2468, 0.2844, 0.2506, 0.299, 0.2794, 0.2754, 0.3108, 0.3444, 0.3382, 0.3082, 0.4288, 0.3828, 0.306, 0.37, 0.3422, 0.3058, 0.4022, 0.4562, 0.408, 0.381, 0.314, 0.3374, 0.3666, 0.342, 0.383, 0.4588, 0.2944, 0.2114, 0.2296, 0.2214, 0.3762, 0.2428, 0.3728, 0.2926, 0.347, 0.2308, 0.3886, 0.4846, 0.4502, 0.3922, 0.476, 0.3644, 0.3998, 0.4594, 0.4866, 0.453, 0.2344, 0.2018, 0.2326, 0.2826, 0.2306, 0.237, 0.204, 0.2274, 0.2328, 0.2656, 0.2156, 0.246, 0.234, 0.2748, 0.288, 0.2448, 0.3194, 0.2802, 0.2714, 0.2908, 0.3162, 0.3284, 0.276, 0.2512, 0.2278, 0.2432, 0.2234, 0.3222, 0.2454, 0.2236, 0.3134, 0.3188, 0.277, 0.3322, 0.3818, 0.2744, 0.3162, 0.3182, 0.3026, 0.489, 0.2926, 0.2956, 0.282, 0.2892, 0.3146, 0.2944, 0.2628, 0.304, 0.3516, 0.396, 0.3316, 0.2898, 0.2974, 0.3296, 0.311, 0.3326, 0.3038, 0.3004, 0.3078, 0.2402, 0.2846, 0.3668, 0.4012, 0.3656, 0.3908, 0.3462, 0.3344, 0.3688, 0.368, 0.357, 0.3882, 0.453, 0.404, 0.4242, 0.4232, 0.3612, 0.4266, 0.4028, 0.3844, 0.4274, 0.4368, 0.4858, 0.4128, 0.4526, 0.4392, 0.3886, 0.5018, 0.446, 0.4032, 0.542, 0.426, 0.3342, 0.2256, 0.473, 0.2454, 0.267, 0.3748, 0.2654, 0.4268, 0.2418, 0.2044, 0.209, 0.2582, 0.209, 0.2942, 0.2502, 0.3124, 0.2118, 0.2798, 0.2064, 0.2068, 0.2076, 0.2018, 0.2514, 0.2428, 0.2082, 0.203, 0.2564, 0.2292, 0.2552, 0.2054, 0.2362, 0.2046, 0.264, 0.2416, 0.2054, 0.2242, 0.3178, 0.3424, 0.338, 0.2486, 0.2648, 0.3048, 0.3058, 0.2844, 0.209, 0.2812, 0.2362, 0.2138, 0.2176, 0.213, 0.212, 0.209, 0.266, 0.2, 0.204, 0.2414, 0.2066, 0.2244, 0.2578, 0.2506, 0.206, 0.2032, 0.244, 0.2308, 0.2156, 0.2162, 0.2116, 0.2084, 0.3192, 0.2872, 0.2766, 0.2682, 0.2142, 0.3206, 0.3276, 0.2884, 0.3234, 0.358, 0.341, 0.3328, 0.3196, 0.3332, 0.3656, 0.371, 0.3622, 0.3212, 0.3994, 0.329, 0.3742, 0.395, 0.3716, 0.3652, 0.4434, 0.488, 0.4934, 0.482, 0.4902, 0.4944, 0.4998, 0.4706, 0.5284, 0.4752, 0.494, 0.2438, 0.213, 0.2364, 0.2582, 0.2892, 0.2006, 0.5238, 0.4936, 0.4992, 0.4744, 0.6046, 0.5172, 0.574, 0.5668, 0.5498, 0.5696, 0.2028, 0.2654, 0.2338, 0.2456, 0.2516, 0.2334, 0.225, 0.2516, 0.2422, 0.2476, 0.3876, 0.3118, 0.357, 0.3948, 0.3388, 0.321, 0.2702, 0.278, 0.3262, 0.306, 0.2796, 0.233, 0.2396, 0.2386, 0.2134, 0.2606, 0.2374, 0.2758, 0.305, 0.277, 0.2058, 0.2016, 0.313, 0.2082, 0.2076, 0.203, 0.2568, 0.2922, 0.2616, 0.3456, 0.2852, 0.3012, 0.2576, 0.3666, 0.2812, 0.3064, 0.3358, 0.256, 0.2752, 0.2382, 0.3302, 0.2858, 0.2506, 0.3022, 0.3252, 0.2848, 0.2134, 0.2582, 0.2774, 0.265, 0.242, 0.2688, 0.2102, 0.228, 0.2606, 0.2604, 0.2316, 0.249, 0.2738, 0.3346, 0.3258, 0.3114, 0.2934, 0.2794, 0.2988, 0.2744, 0.3314, 0.2316, 0.2458, 0.2328, 0.228, 0.2514, 0.2252, 0.2296, 0.243, 0.2028, 0.2268, 0.2182, 0.2416, 0.2152, 0.2378, 0.2028, 0.2608, 0.233, 0.2104, 0.2212, 0.2084, 0.2062, 0.2332, 0.264, 0.2532, 0.2418, 0.3238, 0.3402, 0.2094, 0.2968, 0.2484, 0.2472, 0.276, 0.2008, 0.2118, 0.202, 0.2162, 0.2276, 0.216, 0.201, 0.2056, 0.2158, 0.252, 0.2008, 0.2044, 0.2056, 0.2066, 0.2056, 0.2352, 0.2892, 0.2686, 0.309, 0.3032, 0.3578, 0.3144, 0.2706, 0.2972, 0.3034, 0.2778, 0.3392, 0.3242, 0.3478, 0.3266, 0.3682, 0.3214, 0.3182, 0.2884, 0.339, 0.315, 0.2928, 0.3218, 0.3276, 0.3098, 0.329, 0.2858, 0.36, 0.3148, 0.281, 0.2894, 0.2932, 0.259, 0.2088, 0.308, 0.2506, 0.282, 0.2176, 0.2182, 0.2962, 0.2394, 0.2316, 0.2084, 0.2176, 0.2736, 0.2682, 0.237, 0.2126, 0.2932, 0.2486, 0.2344, 0.299, 0.2528, 0.2576, 0.2588, 0.2532, 0.34, 0.2542, 0.2642, 0.24, 0.2142, 0.2216, 0.238, 0.3046, 0.2364, 0.2124, 0.232, 0.2048, 0.2238, 0.2572, 0.2706, 0.207, 0.268]

In [19]:
test_reals = []
for circuit_info in test_dataset:
    test_reals.append(circuit_info['ground_truth_fidelity'])
    # print(circuit_info['ground_truth_fidelity'], predict)
print(np.abs(np.array(test_reals)-np.array(test_predicts)).mean())

0.06777148891824483


In [22]:
quest = np.abs(np.array(test_reals)-np.array(test_predicts)).tolist()

[0.09788522663116456, 0.00014590024948124558, 0.04073843536376953, 0.19622436952590944, 0.043297242832183835, 0.10417906703948976, 0.08614691038131717, 0.11959899425506593, 0.09294338388442991, 0.0168111262321472, 0.03517675881385804, 0.2446668228149414, 0.026682291698455818, 0.028938558292388894, 0.10874646854400638, 0.025993307209014938, 0.05696077518463136, 0.051233170032501274, 0.2138809394836426, 0.12521104469299316, 0.0018953460693359458, 0.019488820362091097, 0.005050617885589609, 0.104386545753479, 0.06629587316513061, 0.00441619377136232, 0.21613157196044924, 0.018183299160003663, 0.11322074747085575, 0.0675870717048645, 0.08782228555679317, 0.11702114315032963, 0.015887434291839586, 0.09642017049789431, 0.0016966352462768342, 0.024170424556732184, 0.02113486299514772, 0.07595278987884524, 0.0040430885314941545, 0.0022431242942810292, 0.005330212879180896, 0.12456185312271117, 0.0901336002349854, 0.023784341144561794, 0.16761581401824954, 0.044091225528717026, 0.01510099666118

In [ ]:
with open('quest.pkl','wb')as f:
    pickle.dump(quest,f)